# Optimization of an entire influena build for H1, H3 and IBV to include segment and genomes

# Starting Dataset

- A fasta file containing H1, H3 and IBV consensus sequences with the following delimiters `JHID_Segment#` e.g. JH1234_1
- Metadata manually curated containing the JHID, run ID and date_sequenced (inferred for time-resolved analysis)


# Concatenate sequences with vaccine GISAID data

In [3]:
!cat ../data/cat_IV23Run6toIV24Run11.fasta ../data/references.fasta > ../data/sequences.fasta

# Remove sequneces which are out of molecular clock bounds

In [4]:
!seqkit grep -f ../config/exclude.tsv -v ../data/sequences.fasta > ../data/include.fasta

[INFO] 32 patterns loaded from file


## Generate fasta with type and subtype headers 

input vaccine strain headers renamed to the following format:
- Vaccine strains (GISAID): `Isolate name_Segment number` <- this is rediculous but works for now
  

run from the flusort scripts directory for access. flusort will append fasta headers with the following information: 
- `original-seq-name|jhid|segment|type|HA_subtype|NA_subtype|genome_completeness`

In [16]:
cd scripts/flusort/scripts

python flusort.py \
    -i ../../../data/include.fasta \
    -o ../../../results/flusort_out

cd ../../../

zsh:cd:1: no such file or directory: scripts/flusort/scripts
python: can't open file '/Users/elgin/Library/CloudStorage/OneDrive-JohnsHopkins/01_Pekosz_Lab/01_Project_Notebooks/Project_0_Survalience/01_Pekosz_Lab_Survallience/01_Pekosz_Lab_Seasons/2023-24/IBV_Mostafa_Analysis/07_Run6to11_Analysis_nextstrain/scripts/flusort.py': [Errno 2] No such file or directory


# Augur Parse - generate starting metadata

In [5]:
import subprocess

def run_augur_parse():
    command = [
        "augur", "parse",
        "--sequences", "../results/flusort_out/fasta_with_subtype.fasta",
        "--fields", "name", "jhid", "segment", "type", "HA_subtype", "NA_subtype", "genome_completeness",
        "--output-sequences", "../results/sequences.fasta",
        "--output-metadata", "../results/metadata.tsv"
    ]
    
    # Run the command and capture the result
    result = subprocess.run(command, capture_output=True, text=True)
    
    # Print the result and indicate the status
    if result.returncode == 0:
        print(result.stdout)
    else:
        print("Failed to run augur parse command. Error:")
        print(result.stderr)

if __name__ == "__main__":
    run_augur_parse()

# Append dates to starting metadata

In [6]:
import pandas as pd

# File paths
mostafa_meta_file = '../data/IV23Run6toIV24Run11_mostafa_meta.txt'
metadata_file = '../results/metadata.tsv'

# Load the data
mostafa_meta_df = pd.read_csv(mostafa_meta_file, sep='\t')
metadata_df = pd.read_csv(metadata_file, sep='\t')

# Print the columns of each dataframe to check for 'jhid'
print("Columns in mostafa_meta_df:", mostafa_meta_df.columns)
print("Columns in metadata_df:", metadata_df.columns)

# Check if 'jhid' is in both dataframes
if 'jhid' in mostafa_meta_df.columns and 'jhid' in metadata_df.columns:
    # Merge the dataframes on 'jhid'
    merged_df = pd.merge(metadata_df, mostafa_meta_df, on='jhid', how='left')

    # Save the merged dataframe back to the metadata file
    merged_df.to_csv(metadata_file, sep='\t', index=False)

    print("Dataframes merged successfully.")
else:
    print("Error: 'jhid' column not found in one or both dataframes.")


Columns in mostafa_meta_df: Index(['jhid', 'de-id', 'sequencing_run', 'date'], dtype='object')
Columns in metadata_df: Index(['name', 'jhid', 'segment', 'type', 'HA_subtype', 'NA_subtype',
       'genome_completeness'],
      dtype='object')
Dataframes merged successfully.


# Sperate genomes by subtype

In [7]:
mkdir results results/vic results/h1n1 results/h3n2 auspice

In [8]:
"""
Loop through each fasta and sort into type and subtype directories
"""

import subprocess
import os

# Base command for augur filter
base_command = [
    "augur", "filter",
    "--sequences", "../results/sequences.fasta",
    "--metadata", "../results/metadata.tsv"
]

# Queries and their respective outputs
queries_and_outputs = [
    ("type == 'InfluenzaB'", "../results/vic/sequences.fasta", "../results/vic/metadata.tsv"),
    ("HA_subtype == 'H3'", "../results/h3n2/sequences.fasta", "../results/h3n2/metadata.tsv"),
    ("HA_subtype == 'H1'", "../results/h1n1/sequences.fasta", "../results/h1n1/metadata.tsv")
]

# Check and create directories if they do not exist
for _, output_sequences, output_metadata in queries_and_outputs:
    output_dir = os.path.dirname(output_sequences)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

# Run the augur filter command for each query and output
for query, output_sequences, output_metadata in queries_and_outputs:
    command = base_command + ["--query", query, "--output-sequences", output_sequences, "--output-metadata", output_metadata]
    subprocess.run(command)

Note: You did not provide a sequence index, so Augur will generate one. You can generate your own index ahead of time with `augur index` and pass it with `augur filter --sequence-index`.
4316 strains were dropped during filtering
	4316 were filtered out by the query: "type == 'InfluenzaB'"
1274 strains passed all filters
Note: You did not provide a sequence index, so Augur will generate one. You can generate your own index ahead of time with `augur index` and pass it with `augur filter --sequence-index`.
4262 strains were dropped during filtering
	4262 were filtered out by the query: "HA_subtype == 'H3'"
1328 strains passed all filters
Note: You did not provide a sequence index, so Augur will generate one. You can generate your own index ahead of time with `augur index` and pass it with `augur filter --sequence-index`.
2792 strains were dropped during filtering
	2792 were filtered out by the query: "HA_subtype == 'H1'"
2798 strains passed all filters


# Split subtype segments

Since the mostafa lab's pipline designates IBV segments 1 and 2 according to NCBI's numbering, we need to modify it so that it is congruent with the IAV number: 
- IBV NCBI Numbering: 
  - PB1 = Segment 1
  - PB2 = Segment 2
- IAV NCBI Numbering:
  - PB2 = Segment 1
  - PB1 = Segment 2

In [9]:
import subprocess
import os

# Subtypes and their respective directories
subtypes = {
    "h1n1": "../results/h1n1",
    "h3n2": "../results/h3n2",
    "vic": "../results/vic"
}

# Segment names mapping with letter annotations for h1n1 and h3n2
segment_names_h1n1_h3n2 = {
    "1": "pb2",
    "2": "pb1",
    "3": "pa",
    "4": "ha",
    "5": "np",
    "6": "na",
    "7": "mp",
    "8": "ns"
}

# Segment names mapping with letter annotations for vic
segment_names_vic = {
    "1": "pb1",
    "2": "pb2",
    "3": "pa",
    "4": "ha",
    "5": "np",
    "6": "na",
    "7": "mp",
    "8": "ns"
}

def strip_segment_number(fasta_file):
    with open(fasta_file, 'r') as file:
        lines = file.readlines()
    
    with open(fasta_file, 'w') as file:
        for line in lines:
            if line.startswith('>'):
                line = line.split('_')[0] + '\n'
            file.write(line)

def modify_metadata(metadata_file):
    with open(metadata_file, 'r') as file:
        lines = file.readlines()
    
    header = lines[0].strip().split('\t')
    name_index = header.index('name')
    jhid_index = header.index('jhid')
    
    # Create new header with 'jhid' renamed to 'name' and 'name' column removed
    new_header = header[:name_index] + header[name_index+1:jhid_index] + ['name'] + header[jhid_index+1:]
    
    with open(metadata_file, 'w') as file:
        file.write('\t'.join(new_header) + '\n')
        for line in lines[1:]:
            columns = line.strip().split('\t')
            new_columns = columns[:name_index] + columns[name_index+1:jhid_index] + [columns[jhid_index]] + columns[jhid_index+1:]
            file.write('\t'.join(new_columns) + '\n')

# Loop through each subtype
for subtype, subtype_dir in subtypes.items():
    if subtype in ["h1n1", "h3n2"]:
        segment_names = segment_names_h1n1_h3n2
    else:  # vic
        segment_names = segment_names_vic

    for segment_num, segment_name in segment_names.items():
        # Create a new folder for each segment within the subtype directory
        segment_folder = os.path.join(subtype_dir, segment_name)
        if not os.path.exists(segment_folder):
            os.makedirs(segment_folder)

        # Construct the command for augur filter
        command = [
            "augur", "filter",
            "--sequences", f"{subtype_dir}/sequences.fasta",
            "--metadata", f"{subtype_dir}/metadata.tsv",
            "--query", f"segment == {segment_num}",
            "--output-sequences", f"{segment_folder}/sequences.fasta",
            "--output-metadata", f"{segment_folder}/metadata.tsv"
        ]

        # Run augur filter command
        subprocess.run(command)

        # Strip the segment number from the fasta headers
        strip_segment_number(f"{segment_folder}/sequences.fasta")

        # Modify the metadata file
        modify_metadata(f"{segment_folder}/metadata.tsv")

Note: You did not provide a sequence index, so Augur will generate one. You can generate your own index ahead of time with `augur index` and pass it with `augur filter --sequence-index`.
2465 strains were dropped during filtering
	2465 were filtered out by the query: "segment == 1"
333 strains passed all filters
Note: You did not provide a sequence index, so Augur will generate one. You can generate your own index ahead of time with `augur index` and pass it with `augur filter --sequence-index`.
2463 strains were dropped during filtering
	2463 were filtered out by the query: "segment == 2"
335 strains passed all filters
Note: You did not provide a sequence index, so Augur will generate one. You can generate your own index ahead of time with `augur index` and pass it with `augur filter --sequence-index`.
2488 strains were dropped during filtering
	2488 were filtered out by the query: "segment == 3"
310 strains passed all filters
Note: You did not provide a sequence index, so Augur will 

# running nextclade on each of the 8 segments provided 

Only need to download datasets once. Manually rename the downloaded h1n1pdm09 and h3n2 datasets to simplified segment names: `ha` and `na`

In [ ]:
# download nextclade datasets

import subprocess

# Define datasets for h3n2 and h1n1pdm
h3n2_datasets = [
    "flu_h3n2_ha",
    "flu_h3n2_na",
    "nextstrain/flu/h3n2/pb2",
    "nextstrain/flu/h3n2/pb1",
    "nextstrain/flu/h3n2/pa",
    "nextstrain/flu/h3n2/np",
    "nextstrain/flu/h3n2/mp",
    "nextstrain/flu/h3n2/ns"
]

h1n1_datasets = [
    "flu_h1n1pdm_ha",
    "flu_h1n1pdm_na",
    "nextstrain/flu/h1n1pdm/pb2",
    "nextstrain/flu/h1n1pdm/pb1",
    "nextstrain/flu/h1n1pdm/pa",
    "nextstrain/flu/h1n1pdm/np",
    "nextstrain/flu/h1n1pdm/mp",
    "nextstrain/flu/h1n1pdm/ns"
]

# Function to execute nextclade command
def fetch_dataset(dataset, output_dir):
    subprocess.run(["nextclade", "dataset", "get", "-n", dataset, "-o", output_dir])

# Fetch and organize h3n2 datasets
for dataset in h3n2_datasets:
    output_dir = f"../nextclade/flu/h3n2/{dataset.split('/')[-1]}"
    fetch_dataset(dataset, output_dir)

# Fetch and organize h1n1pdm datasets
for dataset in h1n1_datasets:
    output_dir = f"../nextclade/flu/h1n1pdm/{dataset.split('/')[-1]}"
    fetch_dataset(dataset, output_dir)


## Assign clades with nextclade.

We'll run nextclade to assign clades for HA segments and assign quality metrics for the remaining 7 segments. Output to each 'results/{subtype}' folder

In [ ]:
import subprocess
import os

# Define the subtypes and their segments
subtypes = {
    "h3n2": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "h1n1": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "vic": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"]
}

def run_nextclade(subtype, segment):
    input_dir = f"../nextclade/flu/{subtype}/{segment}"
    output_dir = f"../results/{subtype}/{segment}"
    fasta_file = f"{output_dir}/sequences.fasta"
    
    # Create the output directory if it does not exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Run the nextclade command
    subprocess.run(["nextclade", "run", "-D", input_dir, "-O", output_dir, fasta_file])
    
    # Strip the seqName suffix in the nextstrain.tsv file
    nextstrain_file = f"{output_dir}/nextclade.tsv"
    strip_seqName_suffix(nextstrain_file)

def strip_seqName_suffix(nextstrain_file):
    with open(nextstrain_file, 'r') as file:
        lines = file.readlines()

    header = lines[0].strip().split('\t')
    seqName_index = header.index('seqName')

    with open(nextstrain_file, 'w') as file:
        file.write(lines[0])
        for line in lines[1:]:
            columns = line.strip().split('\t')
            columns[seqName_index] = columns[seqName_index].split('_')[0]
            file.write('\t'.join(columns) + '\n')

# Loop through each subtype and segment
for subtype, segments in subtypes.items():
    for segment in segments:
        run_nextclade(subtype, segment)


# Append HA clade to metadata 
I want the clade and subclade columns from the  `results/{subtype}/ha/nextclade.tsv` to be append to each segment's metadata table located at `results/{subtype}/{segment}/metadata.tsv joining by the "seqName" column in the nextclade.tsv table and the "name" column in the metadata.tsv column

2 columns to be made to each meatadata.tsv table
- clade
- subclade


Finally, each segment's metadata file will have ha clade and subclade calls as a trait layer in the build as well as quality data for pre-build filtering. 

In [ ]:
"""
TODO: This can only be run once as further joins do not append.
"""

import os
import pandas as pd

# Define the subtypes and their segments
subtypes = {
    "h3n2": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "h1n1": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "vic": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"]
}

def append_clade_info(subtype, segment):
    ha_nextclade_file = f"../results/{subtype}/ha/nextclade.tsv"
    segment_metadata_file = f"../results/{subtype}/{segment}/metadata.tsv"

    # Load the nextclade data, selecting only 'seqName', 'clade', and 'subclade' columns
    nextclade_df = pd.read_csv(ha_nextclade_file, sep='\t', usecols=['seqName', 'clade', 'subclade'])

    # Check if required columns exist in nextclade.tsv
    required_columns = ['seqName', 'clade', 'subclade']
    for col in required_columns:
        if col not in nextclade_df.columns:
            print(f"Error: Required column '{col}' not found in {ha_nextclade_file}")
            return

    # Load the segment metadata
    metadata_df = pd.read_csv(segment_metadata_file, sep='\t')

    # Merge the dataframes on 'seqName' and 'name'
    merged_df = pd.merge(metadata_df, nextclade_df, left_on='name', right_on='seqName', how='left')

    # Debugging print to check merge output
    print(f"Merged dataframe for {subtype}/{segment}:")
    print(merged_df.head())

    # Check if merge was successful
    if merged_df.empty:
        print(f"No matching data found for {subtype}/{segment}")
        return

    # Update or add the 'clade' and 'subclade' columns in metadata_df
    metadata_df['clade'] = merged_df['clade']
    metadata_df['subclade'] = merged_df['subclade']

    # Save the updated metadata
    metadata_df.to_csv(segment_metadata_file, sep='\t', index=False)

    print(f"Clade information successfully appended to {subtype}/{segment}")

# Loop through each subtype and segment
for subtype, segments in subtypes.items():
    for segment in segments:
        append_clade_info(subtype, segment)

## Merge quality metrics for each segment

In [12]:
import os
import pandas as pd

# Define the subtypes and their segments
subtypes = {
    "h3n2": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "h1n1": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "vic": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"]
}

def append_clade_and_quality_info(subtype, segment):
    segment_nextclade_file = f"../results/{subtype}/{segment}/nextclade.tsv"
    segment_metadata_file = f"../results/{subtype}/{segment}/metadata.tsv"

    # Load the nextclade data, selecting required columns
    nextclade_df = pd.read_csv(segment_nextclade_file, sep='\t', usecols=['seqName', 'qc.overallScore', 'qc.overallStatus', 'coverage'])

    # Check if required columns exist in nextclade.tsv
    required_columns = ['seqName', 'qc.overallScore', 'qc.overallStatus', 'coverage']
    for col in required_columns:
        if col not in nextclade_df.columns:
            print(f"Error: Required column '{col}' not found in {segment_nextclade_file}")
            return

    # Load the segment metadata
    metadata_df = pd.read_csv(segment_metadata_file, sep='\t')

    # Merge the dataframes on 'seqName' and 'name'
    merged_df = pd.merge(metadata_df, nextclade_df, left_on='name', right_on='seqName', how='left')

    # Check if merge was successful
    if merged_df.empty:
        print(f"No matching data found for {subtype}/{segment}")
        return

    # Update or add the columns in metadata_df
    for col in ['qc.overallScore', 'qc.overallStatus', 'coverage']:
        metadata_df[col] = merged_df[col]

    # Drop the 'seqName' column after merging, if it exists
    if 'seqName' in metadata_df.columns:
        metadata_df = metadata_df.drop(columns=['seqName'])

    # Save the updated metadata
    metadata_df.to_csv(segment_metadata_file, sep='\t', index=False)

    print(f"Clade and quality information successfully appended to {subtype}/{segment}")

# Loop through each subtype and segment
for subtype, segments in subtypes.items():
    for segment in segments:
        append_clade_and_quality_info(subtype, segment)

Clade and quality information successfully appended to h3n2/pb2
Clade and quality information successfully appended to h3n2/pb1
Clade and quality information successfully appended to h3n2/pa
Clade and quality information successfully appended to h3n2/ha
Clade and quality information successfully appended to h3n2/np
Clade and quality information successfully appended to h3n2/na
Clade and quality information successfully appended to h3n2/mp
Clade and quality information successfully appended to h3n2/ns
Clade and quality information successfully appended to h1n1/pb2
Clade and quality information successfully appended to h1n1/pb1
Clade and quality information successfully appended to h1n1/pa
Clade and quality information successfully appended to h1n1/ha
Clade and quality information successfully appended to h1n1/np
Clade and quality information successfully appended to h1n1/na
Clade and quality information successfully appended to h1n1/mp
Clade and quality information successfully appended

# Begin nextstrain build for all 8 segments

- this build will use size-based filtering as a crude genome quality metric assuming all consensus sequences are of high quality.

- Add the vaccine strains to each build ✅
- Append HA clade to segment metadata ✅
- Append QC to segment metadata ✅
- Append dates to segment metadata ⚠️ <- you can join this early after the first parse from flusort output. 

nextstrain construction:

1.  filter quality 
    -  coverage 
    -  qc.overallStatus - good or mediocre? likely just good
2. index 
3. align
4. build tree
5. refine tree
6. annotate
7. infer ancestral sequences
8. translate to amino acid mutations
9. calculate tip frequencies
10. export genome builds


## Filter

In [13]:
import os
import subprocess

# Define the subtypes and their segments
subtypes = {
    "h3n2": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "h1n1": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "vic": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"]
}

# Define the minimum lengths for each segment
min_lengths = {
    "pb2": 2000,
    "pb1": 2000,
    "pa": 1800,
    "ha": 1400,
    "np": 1200,
    "na": 1200,
    "mp": 700,
    "ns": 700
}

for subtype, segments in subtypes.items():
    for segment in segments:
        sequences_file = f"../results/{subtype}/{segment}/sequences.fasta"
        metadata_file = f"../results/{subtype}/{segment}/metadata.tsv"
        output_sequences_file = f"../results/{subtype}/{segment}/filtered.fasta"
        output_metadata_file = f"../results/{subtype}/{segment}/filtered.tsv"

        min_length = min_lengths.get(segment, 0)  # Default to 0 if segment not found

        command = [
            "augur", "filter",
            "--sequences", sequences_file,
            "--metadata", metadata_file,
            "--query", "(coverage >= 0.9) & (`qc.overallStatus` == 'good')",  # Add qc_overallStatus == 'mediocre' if needed
            "--min-length", str(min_length),
            "--output-sequences", output_sequences_file,
            "--output-metadata", output_metadata_file
        ]

        subprocess.run(command, check=True)

Note: You did not provide a sequence index, so Augur will generate one. You can generate your own index ahead of time with `augur index` and pass it with `augur filter --sequence-index`.
12 strains were dropped during filtering
	12 were filtered out by the query: "(coverage >= 0.9) & (`qc.overallStatus` == 'good')"
152 strains passed all filters
Note: You did not provide a sequence index, so Augur will generate one. You can generate your own index ahead of time with `augur index` and pass it with `augur filter --sequence-index`.
145 strains were dropped during filtering
	145 were filtered out by the query: "(coverage >= 0.9) & (`qc.overallStatus` == 'good')"
5 strains passed all filters
Note: You did not provide a sequence index, so Augur will generate one. You can generate your own index ahead of time with `augur index` and pass it with `augur filter --sequence-index`.
16 strains were dropped during filtering
	16 were filtered out by the query: "(coverage >= 0.9) & (`qc.overallStatus`

## index 

In [14]:
import subprocess
import os

# Define the subtypes and their segments
subtypes = {
    "h3n2": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "h1n1": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "vic": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"]
}

def index_sequences(subtype, segment):
    sequences_file = f"../results/{subtype}/{segment}/filtered.fasta"
    output_index_file = f"../results/{subtype}/{segment}/filtered.index.tsv"
    
    # Construct the augur index command
    command = [
        "augur", "index",
        "--sequences", sequences_file,
        "--output", output_index_file
    ]
    
    # Run the command
    subprocess.run(command)

# Loop through each subtype and segment
for subtype, segments in subtypes.items():
    for segment in segments:
        index_sequences(subtype, segment)


## align

In [15]:
import subprocess
import os

# Define the subtypes and their segments
subtypes = {
    "h3n2": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "h1n1": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "vic": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"]
}

def align_sequences(subtype, segment):
    sequences_file = f"../results/{subtype}/{segment}/filtered.fasta"
    reference_file = f"../config/{subtype}/reference_{segment}.gb"
    output_aligned_file = f"../results/{subtype}/{segment}/aligned.fasta"
    
    # Construct the augur align command
    command = [
        "augur", "align",
        "--sequences", sequences_file,
        "--nthreads", "8",
        "--reference-sequence", reference_file,
        "--remove-reference",
        "--output", output_aligned_file,
        "--fill-gaps"
    ]
    
    # Run the command
    subprocess.run(command)

# Loop through each subtype and segment
for subtype, segments in subtypes.items():
    for segment in segments:
        align_sequences(subtype, segment)



using mafft to align via:
	mafft --reorder --anysymbol --nomemsave --adjustdirection --thread 8 ../results/h3n2/pb2/aligned.fasta.to_align.fasta 1> ../results/h3n2/pb2/aligned.fasta 2> ../results/h3n2/pb2/aligned.fasta.log 

	Katoh et al, Nucleic Acid Research, vol 30, issue 14
	https://doi.org/10.1093%2Fnar%2Fgkf436

No gaps in alignment to trim (with respect to the reference, NC_007373.1)

using mafft to align via:
	mafft --reorder --anysymbol --nomemsave --adjustdirection --thread 8 ../results/h3n2/pb1/aligned.fasta.to_align.fasta 1> ../results/h3n2/pb1/aligned.fasta 2> ../results/h3n2/pb1/aligned.fasta.log 

	Katoh et al, Nucleic Acid Research, vol 30, issue 14
	https://doi.org/10.1093%2Fnar%2Fgkf436

No gaps in alignment to trim (with respect to the reference, NC_007372.1)

using mafft to align via:
	mafft --reorder --anysymbol --nomemsave --adjustdirection --thread 8 ../results/h3n2/pa/aligned.fasta.to_align.fasta 1> ../results/h3n2/pa/aligned.fasta 2> ../results/h3n2/pa/aligned

## raw tree

In [16]:
import subprocess
import os

# Define the subtypes and their segments
subtypes = {
    "h3n2": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "h1n1": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "vic": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"]
}

def build_raw_tree(subtype, segment):
    alignment_file = f"../results/{subtype}/{segment}/aligned.fasta"
    output_tree_file = f"../results/{subtype}/{segment}/tree_raw.nwk"
    
    # Construct the augur tree command
    command = [
        "augur", "tree",
        "--alignment", alignment_file,
        "--output", output_tree_file
    ]
    
    # Run the command and capture the result
    result = subprocess.run(command, capture_output=True, text=True)
    
    # Print the result and indicate the segment processing status
    if result.returncode == 0:
        print(f"Successfully aligned {segment} for subtype {subtype}. Tree saved to {output_tree_file}")
    else:
        print(f"Failed to align {segment} for subtype {subtype}. Error: {result.stderr}")

# Loop through each subtype and segment
for subtype, segments in subtypes.items():
    for segment in segments:
        print(f"Processing alignment for {segment} in subtype {subtype}...")
        build_raw_tree(subtype, segment)

Processing alignment for pb2 in subtype h3n2...
Successfully aligned pb2 for subtype h3n2. Tree saved to ../results/h3n2/pb2/tree_raw.nwk
Processing alignment for pb1 in subtype h3n2...
Successfully aligned pb1 for subtype h3n2. Tree saved to ../results/h3n2/pb1/tree_raw.nwk
Processing alignment for pa in subtype h3n2...
Successfully aligned pa for subtype h3n2. Tree saved to ../results/h3n2/pa/tree_raw.nwk
Processing alignment for ha in subtype h3n2...
Successfully aligned ha for subtype h3n2. Tree saved to ../results/h3n2/ha/tree_raw.nwk
Processing alignment for np in subtype h3n2...
Successfully aligned np for subtype h3n2. Tree saved to ../results/h3n2/np/tree_raw.nwk
Processing alignment for na in subtype h3n2...
Successfully aligned na for subtype h3n2. Tree saved to ../results/h3n2/na/tree_raw.nwk
Processing alignment for mp in subtype h3n2...
Successfully aligned mp for subtype h3n2. Tree saved to ../results/h3n2/mp/tree_raw.nwk
Processing alignment for ns in subtype h3n2...
Su

## refine

need to first add the date column to the original metadata parsed from the raw data...

In [39]:
import subprocess
import os

# Define the subtypes and their segments
subtypes = {
    "h3n2": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "h1n1": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "vic": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    
}

def refine_tree(subtype, segment):
    tree_file = f"../results/{subtype}/{segment}/tree_raw.nwk"
    alignment_file = f"../results/{subtype}/{segment}/aligned.fasta"
    metadata_file = f"../results/{subtype}/{segment}/filtered.tsv"
    output_tree_file = f"../results/{subtype}/{segment}/tree.nwk"
    output_node_data_file = f"../results/{subtype}/{segment}/branch_lengths.json"
    
    # Construct the augur refine command
    command = [
        "augur", "refine",
        "--tree", tree_file,
        "--alignment", alignment_file,
        "--metadata", metadata_file,
        "--output-tree", output_tree_file,
        "--output-node-data", output_node_data_file,
        "--timetree",
        "--coalescent", "opt",
        "--date-confidence",
        "--date-inference", "marginal",
        "--clock-filter-iqd", "4" # removes tips that deviate more than n_iqd interquartile ranges from the root-to-tip vs time regression.
    ]
    # Add --clock-rate argument only for vic subtype and PB2, NP segments. These are taken from the bedford lab 60yr vic build: 
    # https://nextstrain.org/groups/blab/flu/seasonal/vic/np/60y?l=clock

    if subtype == "vic" and segment == "pb2":
        command.extend(["--clock-rate", "0.00108"])
    if subtype == "vic" and segment == "np":
        command.extend(["--clock-rate", "0.00134"])
    if subtype == "vic" and segment == "na":
        command.extend(["--clock-rate", "0.00133"])
    if subtype == "h3n2" and segment == "pb2":
        command.extend(["--clock-rate", "0.00218"])
    
    # Print progress message
    print(f"Refining tree for {subtype}/{segment}...")
    
    # Run the command
    result = subprocess.run(command, capture_output=True, text=True)
    
    # Check for errors
    if result.returncode != 0:
        print(f"Error refining tree for {subtype}/{segment}:")
        print(result.stderr)
    else:
        print(f"Completed refining tree for {subtype}/{segment}")

# Loop through each subtype and segment
for subtype, segments in subtypes.items():
    for segment in segments:
        refine_tree(subtype, segment)


Refining tree for vic/mp...
Completed refining tree for vic/mp


## annotate 

augur traits \
  --tree ../results/{subtype}/{segment}/tree_raw.nwk \
  --metadata ../results/{subtype}/{segment}/filtered.tsv  \
  --output-node-data ../results/{subtype}/{segment}/traits.json \
  --columns clade subclade qc_overallStatus qc_overallScore coverage sequencing_run \
  --confidence
  

In [40]:
import subprocess
import os
import time

# Define the subtypes and their segments
subtypes = {
    "h3n2": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "h1n1": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "vic": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"]
}

def annotate_traits(subtype, segment):
    tree_file = f"../results/{subtype}/{segment}/tree.nwk"
    metadata_file = f"../results/{subtype}/{segment}/filtered.tsv"
    output_node_data_file = f"../results/{subtype}/{segment}/traits.json"
    
    # Construct the augur traits command
    command = [
        "augur", "traits",
        "--tree", tree_file,
        "--metadata", metadata_file,
        "--output-node-data", output_node_data_file,
        "--columns", "clade", "subclade", "qc_overallStatus", "qc_overallScore", "coverage", "sequencing_run",
        "--confidence"
    ]
    
    # Print progress message
    print(f"Annotating traits for {subtype}/{segment}...")
    
    # Run the command
    result = subprocess.run(command, capture_output=True, text=True)
    
    # Check for errors
    if result.returncode != 0:
        print(f"Error annotating traits for {subtype}/{segment}:")
        print(result.stderr)
    else:
        print(f"Completed annotating traits for {subtype}/{segment}")

# Timing the execution
start_time = time.time()

# Loop through each subtype and segment
for subtype, segments in subtypes.items():
    for segment in segments:
        annotate_traits(subtype, segment)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Total execution time: {elapsed_time:.2f} seconds")

Annotating traits for vic/mp...
Completed annotating traits for vic/mp
Total execution time: 4.98 seconds


## Infer ancestral sequences

In [41]:
import subprocess
import os
import time

# Define the subtypes and their segments
subtypes = {
    "h3n2": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "h1n1": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "vic": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"]
}

def run_augur_ancestral(subtype, segment):
    tree_file = f"../results/{subtype}/{segment}/tree.nwk"
    alignment_file = f"../results/{subtype}/{segment}/aligned.fasta"
    output_node_data = f"../results/{subtype}/{segment}/nt_muts.json"
    
    # Augur ancestral command
    command = [
        "augur", "ancestral",
        "--tree", tree_file,
        "--alignment", alignment_file,
        "--output-node-data", output_node_data,
        "--inference", "joint"
    ]
    
    # Print progress message
    print(f"Running augur ancestral for {subtype}/{segment}...")
    
    # Run the command
    result = subprocess.run(command, capture_output=True, text=True)
    
    # Check for errors
    if result.returncode != 0:
        print(f"Error running augur ancestral for {subtype}/{segment}:")
        print(result.stderr)
    else:
        print(f"Completed augur ancestral for {subtype}/{segment}")

# Timing the execution
start_time = time.time()

# Loop through each subtype and segment
for subtype, segments in subtypes.items():
    for segment in segments:
        run_augur_ancestral(subtype, segment)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Total execution time: {elapsed_time:.2f} seconds")

Running augur ancestral for vic/mp...
Completed augur ancestral for vic/mp
Total execution time: 1.67 seconds


## translate 

In [42]:
import subprocess
import time

# Define the subtypes and their segments
subtypes = {
    "h3n2": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "h1n1": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "vic": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"]
}

def translate_sequences(subtype, segment):
    tree_file = f"../results/{subtype}/{segment}/tree.nwk"
    ancestral_sequences_file = f"../results/{subtype}/{segment}/nt_muts.json"
    reference_sequence_file_ha = f"../config/{subtype}/genome_annotation_ha.gff"
    reference_sequence_file = f"../config/{subtype}/reference_{segment}.gb"
    output_node_data_file = f"../results/{subtype}/{segment}/aa_muts.json"
    
    # Use the .gff file for the HA segment and the .gb file for all other segments
    if segment == "ha":
        reference_sequence_file = reference_sequence_file_ha
    
    # Construct the augur translate command
    command = [
        "augur", "translate",
        "--tree", tree_file,
        "--ancestral-sequences", ancestral_sequences_file,
        "--reference-sequence", reference_sequence_file,
        "--output-node-data", output_node_data_file
    ]
    
    # Print progress message
    print(f"Translating sequences for {subtype}/{segment}...")
    
    # Run the command
    result = subprocess.run(command, capture_output=True, text=True)
    
    # Output the result
    print(f"stdout for {subtype}/{segment}:\n{result.stdout}")
    if result.stderr:
        print(f"stderr for {subtype}/{segment}:\n{result.stderr}")

# Timing the execution
start_time = time.time()

# Loop through each subtype and segment
for subtype, segments in subtypes.items():
    for segment in segments:
        translate_sequences(subtype, segment)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Total execution time: {elapsed_time:.2f} seconds")


Translating sequences for vic/mp...
stdout for vic/mp:
Read in 3 features from reference sequence file
Validating schema of '../results/vic/mp/nt_muts.json'...
Validating schema of '../results/vic/mp/aa_muts.json'...
amino acid mutations written to ../results/vic/mp/aa_muts.json

Total execution time: 1.62 seconds


## Export auspice v2 json

In [62]:
import subprocess
import os

# Define the subtypes and their segments

subtypes = {
    "h3n2": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "h1n1": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "vic": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"]
}

def export_build(subtype, segment):
    tree_file = f"../results/{subtype}/{segment}/tree.nwk"
    metadata_file = f"../results/{subtype}/{segment}/filtered.tsv"
    node_data_files = [
        f"../results/{subtype}/{segment}/branch_lengths.json",
        f"../results/{subtype}/{segment}/traits.json",
        f"../results/{subtype}/{segment}/nt_muts.json",
        f"../results/{subtype}/{segment}/aa_muts.json",
        f"../config/{subtype}/vaccine.json" # add updated vaccine strains here.
    ]
    auspice_config_file = f"../config/{subtype}/auspice_config.json"
    output_file = f"../auspice/{subtype}/{segment}.json"
    
    # Ensure output directory exists, create if not
    output_dir = os.path.dirname(output_file)
    os.makedirs(output_dir, exist_ok=True)
    
    # Construct the augur export command
    command = [
        "augur", "export", "v2",
        "--tree", tree_file,
        "--metadata", metadata_file,
        "--node-data"
    ]
    command.extend(node_data_files)
    command.extend([
        "--auspice-config", auspice_config_file,
        "--output", output_file
    ])
    
    print(f"Exporting build for {subtype}/{segment}...")
    
    result = subprocess.run(command, capture_output=True, text=True)
    
    # Check for errors
    if result.returncode != 0:
        print(f"Error exporting build for {subtype}/{segment}:")
        print(result.stderr)
        print(result.stdout)
    else:
        print(f"Completed exporting build for {subtype}/{segment}")

# Loop through each subtype and segment
for subtype, segments in subtypes.items():
    for segment in segments:
        export_build(subtype, segment)

Exporting build for h3n2/pb2...
Completed exporting build for h3n2/pb2
Exporting build for h3n2/pb1...
Completed exporting build for h3n2/pb1
Exporting build for h3n2/pa...
Completed exporting build for h3n2/pa
Exporting build for h3n2/ha...
Completed exporting build for h3n2/ha
Exporting build for h3n2/np...
Completed exporting build for h3n2/np
Exporting build for h3n2/na...
Completed exporting build for h3n2/na
Exporting build for h3n2/mp...
Completed exporting build for h3n2/mp
Exporting build for h3n2/ns...
Completed exporting build for h3n2/ns
Exporting build for h1n1/pb2...
Completed exporting build for h1n1/pb2
Exporting build for h1n1/pb1...
Completed exporting build for h1n1/pb1
Exporting build for h1n1/pa...
Completed exporting build for h1n1/pa
Exporting build for h1n1/ha...
Completed exporting build for h1n1/ha
Exporting build for h1n1/np...
Completed exporting build for h1n1/np
Exporting build for h1n1/na...
Completed exporting build for h1n1/na
Exporting build for h1n1/m

# Uploading builds to Private Nextstrain Instance

build names:
- "title": "Baltimore 2023-24 season influenza A/H1N1pdm evolution"
- "title": "Baltimore 2023-24 season influenza B/Vic evolution"
- "title": "Baltimore 2023-24 season influenza A/H3N2 evolution"

In [63]:
import subprocess

# Define the subtypes and their segments
subtypes = {
    "h3n2": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "h1n1": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "vic": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"]
}

def nextstrain_login():
    # Log in to Nextstrain CLI
    print("Logging in to Nextstrain...")
    result = subprocess.run(["nextstrain", "login"], capture_output=True, text=True)
    
    # Check for errors
    if result.returncode != 0:
        print("Error logging in to Nextstrain:")
        print(result.stderr)
    else:
        print("Successfully logged in to Nextstrain")

def nextstrain_upload(subtype):
    # Construct the nextstrain remote upload command
    command = [
        "nextstrain", "remote", "upload",
        f"nextstrain.org/groups/PekoszLab/{subtype}"
    ] + [f"../auspice/{subtype}/{segment}.json" for segment in subtypes[subtype]]
    
    # Print progress message
    print(f"Uploading data for {subtype} to Nextstrain...")
    
    # Run the command
    result = subprocess.run(command, capture_output=True, text=True)
    
    # Check for errors
    if result.returncode != 0:
        print(f"Error uploading data for {subtype}:")
        print(result.stderr)
    else:
        print(f"Successfully uploaded data for {subtype}")

# Log in to Nextstrain CLI
nextstrain_login()

# Upload data for each subtype
for subtype in subtypes.keys():
    nextstrain_upload(subtype)

Logging in to Nextstrain...
Successfully logged in to Nextstrain
Uploading data for h3n2 to Nextstrain...
Successfully uploaded data for h3n2
Uploading data for h1n1 to Nextstrain...
Successfully uploaded data for h1n1
Uploading data for vic to Nextstrain...
Successfully uploaded data for vic


# Upload Public Build

In [64]:
import subprocess

# Define the subtypes and their segments
subtypes = {
    "h3n2": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "h1n1": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"],
    "vic": ["pb2", "pb1", "pa", "ha", "np", "na", "mp", "ns"]
}

def nextstrain_login():
    # Log in to Nextstrain CLI
    print("Logging in to Nextstrain...")
    result = subprocess.run(["nextstrain", "login"], capture_output=True, text=True)
    
    # Check for errors
    if result.returncode != 0:
        print("Error logging in to Nextstrain:")
        print(result.stderr)
    else:
        print("Successfully logged in to Nextstrain")

def nextstrain_upload(subtype):
    # Construct the nextstrain remote upload command
    command = [
        "nextstrain", "remote", "upload",
        f"nextstrain.org/groups/PekoszLab-Public/{subtype}"
    ] + [f"../auspice/{subtype}/{segment}.json" for segment in subtypes[subtype]]
    
    # Print progress message
    print(f"Uploading data for {subtype} to Nextstrain...")
    
    # Run the command
    result = subprocess.run(command, capture_output=True, text=True)
    
    # Check for errors
    if result.returncode != 0:
        print(f"Error uploading data for {subtype}:")
        print(result.stderr)
    else:
        print(f"Successfully uploaded data for {subtype}")

# Log in to Nextstrain CLI
nextstrain_login()

# Upload data for each subtype
for subtype in subtypes.keys():
    nextstrain_upload(subtype)

Logging in to Nextstrain...
Successfully logged in to Nextstrain
Uploading data for h3n2 to Nextstrain...
Successfully uploaded data for h3n2
Uploading data for h1n1 to Nextstrain...
Successfully uploaded data for h1n1
Uploading data for vic to Nextstrain...
Successfully uploaded data for vic
